In [1]:
import pandas as pd

air_data = pd.read_csv('AirQuality_Daily_StudentVersion.csv')

air_data = pd.DataFrame(air_data)


In [75]:
# Requirement 1 
# 5 locations in Nebraska with the highest mean and median concentrations of VOC, PM 2.5, and PM 10.0 

stats = air_data.groupby("sensor.name").agg(
    voc_mean=("voc", "mean"),
    voc_median=("voc", "median"),
    pm25_mean=("pm2.5_atm", "mean"),
    pm25_median=("pm2.5_atm", "median"),
    pm10_mean=("pm10.0_atm", "mean"),
    pm10_median=("pm10.0_atm", "median")
).reset_index()

top5_voc = stats.sort_values("voc_mean", ascending=False).head(5)
top5_pm25 = stats.sort_values("pm25_mean", ascending=False).head(5)
top5_pm10 = stats.sort_values("pm10_mean", ascending=False).head(5)

top5_vocM = stats.sort_values("voc_median", ascending=False).head(5)
top5_pm25M = stats.sort_values("pm25_median", ascending=False).head(5)
top5_pm10M = stats.sort_values("pm10_median", ascending=False).head(5)

top5_voc
top5_pm25
top5_pm10
top5_vocM
top5_pm25M
top5_pm10M

,sensor.name,voc_mean,voc_median,pm25_mean,pm25_median,pm10_mean,pm10_median
4,Broken Bow,158.285807,138.559646,928.710593,36.050240,929.678512,43.179094
0,#16 - Richardson County Courthouse,86.758650,84.470500,700.127342,11.977344,701.632446,13.305615
6,ELVPHD Norfolk HD 4,360.833744,368.580500,13.369492,9.706229,15.969341,11.474708
2,#18 - Southeast District Health Department- Te...,196.150771,138.582583,613.175352,10.322875,614.227248,11.433729
8,ELVPHD Wisner HD 5,102.291126,100.709271,11.154420,8.464583,14.462116,10.935958


In [79]:
# Requirement 2
# On what days did the maximums occur and where were the maximums 



voc_max = air_data.loc[air_data["voc"].idxmax(), 
                 ["date", "sensor.name", "voc"]]
pm25_max = air_data.loc[air_data["pm2.5_atm"].idxmax(), 
                  ["date", "sensor.name", "pm2.5_atm"]]
pm10_max = air_data.loc[air_data["pm10.0_atm"].idxmax(), 
                  ["date", "sensor.name","pm10.0_atm"]]
voc_max
pm25_max
pm10_max
        



date                                     02/18/25
sensor.name    #16 - Richardson County Courthouse
pm10.0_atm                            3784.682542
Name: 7561, dtype: object

In [85]:
# Requirement 3
# Does humidity and temprature have an effet on the air quality 


air_data["humidity"] = pd.cut(
    air_data["humidity"],
    bins=[0, 50, 80, 100],
    labels=["Low", "High", "Very High"],
    include_lowest=True
)

air_data["temp"] = pd.cut(
    air_data["temperature"],
    bins=[-100, 32, 50, 70, 200],
    labels=["Below Freezing", "Cool", "Warm", "Hot"],
    include_lowest=True
)

humidity_effect = air_data.groupby("humidity")[["voc", "pm2.5_atm", "pm10.0_atm"]].mean()
temperature_effect = air_data.groupby("temp")[["voc", "pm2.5_atm", "pm10.0_atm"]].mean()

humidity_effect


/var/folders/6q/tq9k74791dggcsh6r276hy8w0000gr/T/ipykernel_64740/1543702203.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  humidity_effect = air_data.groupby("humidity")[["voc", "pm2.5_atm", "pm10.0_atm"]].mean()
/var/folders/6q/tq9k74791dggcsh6r276hy8w0000gr/T/ipykernel_64740/1543702203.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  temperature_effect = air_data.groupby("temp")[["voc", "pm2.5_atm", "pm10.0_atm"]].mean()


,voc,pm2.5_atm,pm10.0_atm
humidity,,,
Low,279.128737,76.462835,77.828905
High,254.022478,81.280480,82.972745
Very High,19.395555,5.062575,6.322328


In [95]:
# Reqiorment 4 
# Have there been any AQI health risks associated with any ofthe locations

aqi_risk = air_data[
    (air_data["pm2.5_atm"] >= 35.5) |
    (air_data["pm10.0_atm"] >= 155)]
aqi_events = aqi_risk[[
    "date", "sensor.name", "pm2.5_atm", "pm10.0_atm"
]].sort_values("date")

aqi_events

,date,sensor.name,pm2.5_atm,pm10.0_atm
6565,01/01/25,Broken Bow,2107.029354,2107.215604
6556,01/01/25,#16 - Richardson County Courthouse,2114.623250,2114.860313
6585,01/02/25,Broken Bow,1968.824813,1969.514229
6576,01/02/25,#16 - Richardson County Courthouse,2135.601729,2136.544792
6596,01/03/25,#16 - Richardson County Courthouse,2357.012438,2359.675646
...,...,...,...,...
6496,12/29/24,#16 - Richardson County Courthouse,1433.841104,1436.412979
6525,12/30/24,Broken Bow,1722.934604,1723.087500
6516,12/30/24,#16 - Richardson County Courthouse,1946.299250,1948.080313
6545,12/31/24,Broken Bow,2102.347375,2102.588313


In [81]:
# requiorment 5 
# these the sensor altitude have an effect on the air quality values 

altitude_corr = air_data[[
    "sensor.altitude", "voc", "pm2.5_atm", "pm10.0_atm"
]].corr()

air_data["altitude"] = pd.qcut(air_data["sensor.altitude"], q=3, labels=["Low", "Medium", "High"])

altitude_effect = air_data.groupby("altitude")[["voc", "pm2.5_atm", "pm10.0_atm"]].mean()
altitude_effect


/var/folders/6q/tq9k74791dggcsh6r276hy8w0000gr/T/ipykernel_64740/1645994312.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  altitude_effect = air_data.groupby("altitude")[["voc", "pm2.5_atm", "pm10.0_atm"]].mean()


,voc,pm2.5_atm,pm10.0_atm
altitude,,,
Low,251.336154,116.922252,118.744655
Medium,289.438856,24.152354,25.482386
High,283.223755,147.867249,149.067477
